In [1]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[2] 
 AppName 
 Databricks Shell

In [2]:
file_location1 = "/FileStore/tables/aries_crash_data_2016.csv"
file_location2 = "/FileStore/tables/aries_crash_data_2017.csv"
file_location3 = "/FileStore/tables/aries_crash_data_2018.csv"
file_location4 = "/FileStore/tables/aries_crash_data_2015.csv"

file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","
quotechar = '"'

# The applied options are for CSV files. For other file types, these will be ignored.
df_2015 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("quotechar", quotechar)\
  .load(file_location4)

# The applied options are for CSV files. For other file types, these will be ignored.
df_2016 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("quotechar", quotechar)\
  .load(file_location1)
  
df_2017 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("quotechar", quotechar)\
  .load(file_location2)

df_2018 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("quotechar", quotechar)\
  .load(file_location3)


appended1 = df_2015.union(df_2016)
appended2 = appended1.union(df_2017)
complete_set = appended2.union(df_2018)
#complete_set.count()

df = complete_set
# display(df)
#df.printSchema()

In [3]:
## Reorganize Data
df1 = df.dropDuplicates(['INDIVIDUAL_MR_RECORD'])
df1.count()

Out[3]: 875761

In [4]:
from pyspark.sql import functions as sf
from pyspark.sql.functions import col, avg


In [5]:
from pyspark.sql.functions import unix_timestamp, from_unixtime
labeledData = df1.select(df1['LIGHTCONDDESCR'].alias('light'),df1['WEATHERDESCR'].alias('weather')
                        ,df1['SPEEDLIMITTXT'].alias('speed'),df1['DEADNMB'].alias('dead'),df1['COLLISION_DAY'].alias('weekday')
                       ,df1['COLLISION_TIME'].alias('time'),df1['COLLISION_TIME_AM_PM'].alias('ampm'),df1['LATDECIMALNMB'].alias('lat'),df1['LONGDECIMALNMB'].alias('long'))
## Concatenate time and am/pm
df2 = labeledData.withColumn('collidetime', 
                    sf.concat(sf.col('time'),sf.lit(' '), sf.col('ampm')))
labeledData = df2.drop('time','ampm')
## Convert into time value
df_time_twenfour = labeledData.withColumn('collidetime',
                                      from_unixtime(unix_timestamp(col(('collidetime')), "hh:mm aa"), "HH:mm"))
labeledData = df_time_twenfour

In [6]:
labeledData.createOrReplaceTempView("crash");
# CREATE 2 BUCKETS of DEATH
sqlStatement = """
    SELECT *,
    CASE
     WHEN dead>0 THEN 1 
     WHEN dead=0 THEN 0

    END as death
    FROM crash 
"""
labeled_with_newFeatures = sqlContext.sql(sqlStatement)

In [7]:
from pyspark.sql.types import IntegerType
labeledData = labeled_with_newFeatures.na.drop()

labeledData =labeledData.withColumn("speed", labeledData["speed"].cast(IntegerType()))
labeledData = labeledData.filter(labeledData.speed > 0)
labeledData = labeledData.drop('dead')
#labeledData.take(2)

In [8]:
labeledData.createOrReplaceTempView("crash1")
# CREATE 5 BUCKETS of speed limit
sqlStatement = """
    SELECT *,
    CASE
     WHEN speed < 30 THEN 30
     WHEN speed = 30 THEN 30
     WHEN speed > 30 AND speed < 40 THEN 40
     WHEN speed = 40 THEN 40
     WHEN speed > 40 AND speed < 50 THEN 50
     WHEN speed = 50 THEN 50
     WHEN speed >50 AND speed < 60 THEN 60
     WHEN speed = 60 THEN 60
     WHEN speed > 60 THEN 70
     
    END as speedlim
    FROM crash1 
"""

labeled_with_newFeatures_1 = sqlContext.sql(sqlStatement)

In [9]:
## Drop None values
labeled_with_newFeatures_1.createOrReplaceTempView('crash2')
sqlStatement = """
    SELECT distinct speedlim
    FROM crash2
"""
labeled_2 = sqlContext.sql(sqlStatement)
#labeled_2.collect()

In [10]:
labeledData = labeled_with_newFeatures_1
labeledData = labeledData.drop('speed')
## Drop Unknown values from 'light' variable
labeledData = labeledData.filter(labeledData.light != 'Unknown')

In [11]:
labeledData.take(2)

In [12]:
labeledData.createOrReplaceTempView("crash1")
# CREATE 12 BUCKETS of collision time
sqlStatement = """
    SELECT *,
    CASE
     WHEN collidetime < '02:00' OR collidetime = '02:00' THEN '12-2 AM'
     WHEN (collidetime < '04:00' AND collidetime > '02:00') OR collidetime = '04:00' THEN '2-4 AM'
     WHEN (collidetime < '06:00' AND collidetime > '04:00') OR collidetime = '04:00' THEN '4-6 AM'
     WHEN (collidetime < '08:00' AND collidetime > '06:00') OR collidetime = '04:00' THEN '6-8 AM'
     WHEN (collidetime < '10:00' AND collidetime > '08:00') OR collidetime = '04:00' THEN '8-10 AM'
     WHEN (collidetime < '12:00' AND collidetime > '10:00') OR collidetime = '04:00' THEN '10 AM - 12 PM'
     WHEN (collidetime < '14:00' AND collidetime > '12:00') OR collidetime = '04:00' THEN '12-2 PM'
     WHEN (collidetime < '16:00' AND collidetime > '14:00') OR collidetime = '04:00' THEN '2-4 PM'
     WHEN (collidetime < '18:00' AND collidetime > '16:00') OR collidetime = '04:00' THEN '4-6 PM'
     WHEN (collidetime < '20:00' AND collidetime > '18:00') OR collidetime = '04:00' THEN '6-8 PM'
     WHEN (collidetime < '22:00' AND collidetime > '20:00') OR collidetime = '04:00' THEN '8-10 PM'
     WHEN (collidetime < '24:00' AND collidetime > '22:00') OR collidetime = '04:00' THEN '10 PM - 12 AM'
    END as timecollide
    FROM crash1 
"""
labeled_with_newFeatures_2 = sqlContext.sql(sqlStatement)

In [13]:
labeled_with_newFeatures_2.createOrReplaceTempView('crash2')
sqlStatement = """
    SELECT distinct timecollide
    FROM crash2
"""
labeled_2 = sqlContext.sql(sqlStatement)
#labeled_2.collect()

In [14]:
labeledData = labeled_with_newFeatures_2
labeledData = labeledData.na.drop(subset=["timecollide"])
labeledData = labeledData.drop('collidetime')

In [15]:
from pyspark.sql.functions import *
labeledData = labeledData.withColumn('weekday', rtrim(labeledData.weekday))

In [16]:
labeledData.take(2)

In [17]:
## DONE with variable grouping

In [19]:
# Create a view or table

temp_table_name = "aries"

labeledData.createOrReplaceTempView(temp_table_name)

In [20]:
#mysubset = spark.sql("Select Distinct INDIVIDUAL_MR_RECORD,WEATHERCDE, LIGHTCONDCDE, TIMENOTIFIEDTXT, SPEEDLIMITTXT,LATDECIMALNMB, LONGDECIMALNMB,INJUREDNMB, DEADNMB,LATDECIMALNMB, LONGDECIMALNMB from  `aries` ")

In [21]:
## create blocks for labeledData dataframe

In [22]:
temp_table_name = "mysubsetTable"

labeledData.createOrReplaceTempView(temp_table_name)

In [23]:
def create_blocks():
    MinLt = 37.55
    MaxLt = 41.95
    MinLn = -88.24
    MaxLn = -84.64
    blocks = {}
    Lt1 = MinLt
    Ln1 = MinLn
    n=1
    
    for i in range(30):
        if i == 0:
            Lt1 = Lt1
        else:
            Lt1 = Lt1+0.15
        Ln1=MinLn
        for j in range(24):
            point = []
            #block_coord = []
            Lt2 = Lt1+0.15
            Ln2 = Ln1+0.15
            PCentrx = Lt1+0.1
            PCentry = Ln1+0.1
            point = [n,Lt1,Lt2,Ln1,Ln2,PCentrx,PCentry]
            Ln1 = Ln2
        
            if n not in blocks.keys():
                blocks[n] = []
                blocks[n] = point
            
            n+=1
            
    return blocks
indiana_split = create_blocks()
len(indiana_split)

In [25]:
def create_blocks1():
    MinLt = 37.55
    MaxLt = 41.95
    MinLn = -88.24
    MaxLn = -84.64
    blocks = {}
    Lt1 = MinLt
    Ln1 = MinLn
    n=1
    
    for i in range(110):
        if i == 0:
            Lt1 = Lt1
        else:
            Lt1 = Lt1+0.04
        Ln1=MinLn
        for j in range(90):
            point = []
            #block_coord = []
            Lt2 = Lt1+0.04
            Ln2 = Ln1+0.04
            PCentrx = Lt1+0.02
            PCentry = Ln1+0.02
            point = [n,Lt1,Lt2,Ln1,Ln2,PCentrx,PCentry]
            Ln1 = Ln2
        
            if n not in blocks.keys():
                blocks[n] = []
                blocks[n] = point
            
            n+=1
            
    return blocks
indiana_split1 = create_blocks1()

In [26]:
len(indiana_split1)

In [27]:
## 110 blocks
import pandas as pd
pdf = pd.DataFrame.from_dict(indiana_split1, orient = 'index')
blocks = spark.createDataFrame(pdf)

from pyspark.sql.functions import col
blocks = blocks.select(col("0").alias("Blocks"), col("1").alias("x1"), col("2").alias("x2"), col("3").alias("y1"), col("4").alias("y2"), col("5").alias("Xcentr"), col("6").alias("Ycentr"))
blocks.show()

+------+-----+------------------+------------------+------------------+------+------------------+
Blocks| x1| x2| y1| y2|Xcentr| Ycentr|
+------+-----+------------------+------------------+------------------+------+------------------+
 1|37.55|37.589999999999996| -88.24|-88.19999999999999| 37.57| -88.22|
 2|37.55|37.589999999999996|-88.19999999999999|-88.15999999999998| 37.57|-88.17999999999999|
 3|37.55|37.589999999999996|-88.15999999999998|-88.11999999999998| 37.57|-88.13999999999999|
 4|37.55|37.589999999999996|-88.11999999999998|-88.07999999999997| 37.57|-88.09999999999998|
 5|37.55|37.589999999999996|-88.07999999999997|-88.03999999999996| 37.57|-88.05999999999997|
 6|37.55|37.589999999999996|-88.03999999999996|-87.99999999999996| 37.57|-88.01999999999997|
 7|37.55|37.589999999999996|-87.99999999999996|-87.95999999999995| 37.57|-87.97999999999996|
 8|37.55|37.589999999999996|-87.95999999999995|-87.91999999999994| 37.57|-87.93999999999996|
 9|37.55|37.589999999999996|-87.91999999999994|-87.87999999999994| 37.57|-87.89999999999995|
 10|37.55|37.589999999999996|-87.87999999999994|-87.83999999999993| 37.57|-87.85999999999994|
 11|37.55|37.589999999999996|-87.83999999999993|-87.79999999999993| 37.57|-87.81999999999994|
 12|37.55|37.589999999999996|-87.79999999999993|-87.75999999999992| 37.57|-87.77999999999993|
 13|37.55|37.589999999999996|-87.75999999999992|-87.71999999999991| 37.57|-87.73999999999992|
 14|37.55|37.589999999999996|-87.71999999999991|-87.67999999999991| 37.57|-87.69999999999992|
 15|37.55|37.589999999999996|-87.67999999999991| -87.6399999999999| 37.57|-87.65999999999991|
 16|37.55|37.589999999999996| -87.6399999999999| -87.5999999999999| 37.57| -87.6199999999999|
 17|37.55|37.589999999999996| -87.5999999999999|-87.55999999999989| 37.57| -87.5799999999999|
 18|37.55|37.589999999999996|-87.55999999999989|-87.51999999999988| 37.57|-87.53999999999989|
 19|37.55|37.589999999999996|-87.51999999999988|-87.47999999999988| 37.57|-87.49999999999989|
 20|37.55|37.589999999999996|-87.47999999999988|-87.43999999999987| 37.57|-87.45999999999988|
+------+-----+------------------+------------------+------------------+------+------------------+
only showing top 20 rows

In [28]:
# export blocks to csv file
#blocks.toPandas().to_csv('/dbfs/FileStore/blocks.csv')

In [29]:
temp_table_name_1 = "mysubsetTable"

labeledData.createOrReplaceTempView(temp_table_name_1)

temp_table_name = "blockstable"

blocks.createOrReplaceTempView(temp_table_name)

In [30]:
results = spark.sql("Select M.*,B.Blocks from mysubsetTable M, blockstable B where M.lat >= B.x1 and M.lat < B.x2 and M.long >= B.Y1 and M.long < B.Y2")

In [31]:
#results.show()

In [32]:
#results

Out[64]: DataFrame[light: string, weather: string, weekday: string, lat: string, long: string, death: int, speedlim: int, timecollide: string, Blocks: bigint]

In [33]:
df_groupby = results.groupby('Blocks','speedlim','weekday','light','timecollide','weather').agg(sf.count('Blocks').alias("frequency"))
#df_groupby.show()

In [34]:
df_groupby.sort(col('frequency').desc()).show()

+------+--------+---------+--------+-----------+-------+---------+
Blocks|speedlim| weekday| light|timecollide|weather|frequency|
+------+--------+---------+--------+-----------+-------+---------+
 7998| 30| THURSDAY|Daylight| 4-6 PM| Clear| 98|
 7998| 30|WEDNESDAY|Daylight| 4-6 PM| Clear| 89|
 7998| 30| MONDAY|Daylight| 4-6 PM| Clear| 88|
 7998| 30| FRIDAY|Daylight| 2-4 PM| Clear| 85|
 5003| 30| TUESDAY|Daylight| 4-6 PM| Clear| 83|
 3643| 30| FRIDAY|Daylight| 12-2 PM| Clear| 81|
 6424| 30| THURSDAY|Daylight| 2-4 PM| Clear| 81|
 6424| 30| THURSDAY|Daylight| 4-6 PM| Clear| 81|
 6424| 30| FRIDAY|Daylight| 4-6 PM| Clear| 79|
 7998| 30| TUESDAY|Daylight| 4-6 PM| Clear| 77|
 7998| 30| FRIDAY|Daylight| 4-6 PM| Clear| 75|
 5093| 40| THURSDAY|Daylight| 4-6 PM| Clear| 75|
 5093| 40| MONDAY|Daylight| 4-6 PM| Clear| 75|
 5003| 30| THURSDAY|Daylight| 4-6 PM| Clear| 74|
 5003| 30| MONDAY|Daylight| 4-6 PM| Clear| 74|
 6424| 30|WEDNESDAY|Daylight| 4-6 PM| Clear| 72|
 6424| 30| TUESDAY|Daylight| 4-6 PM| Clear| 72|
 8088| 40| MONDAY|Daylight| 4-6 PM| Clear| 72|
 7998| 30| TUESDAY|Daylight| 2-4 PM| Clear| 72|
 3643| 30| FRIDAY|Daylight| 2-4 PM| Clear| 72|
+------+--------+---------+--------+-----------+-------+---------+
only showing top 20 rows

In [35]:
temp_table_name = "frequency"

df_groupby.createOrReplaceTempView(temp_table_name)

In [36]:
result1 = spark.sql("Select F.*,B.x1, B.x2,B.y1,B.y2,B.Xcentr,B.Ycentr from frequency as F, blockstable B where F.Blocks = B.Blocks")

In [37]:
result1.toPandas().to_csv('/dbfs/FileStore/aries_accidents_2.csv')

In [38]:
#result1.show()
#result1.sort(col('frequency').desc()).show()

In [39]:
#result1.count()

In [40]:
## Create NO ACCIDENT DATA
import itertools


In [41]:
blk_lst = []
for i in range(1,721):
  blk_lst.append(i)

a = [blk_lst,[30,40,50,60,70],['MONDAY','TUESDAY','WEDNESDAY','THURSDAY','FRIDAY','SATURDAY','SUNDAY'],['Dark (Not Lighted)','Dark (Lighted)','Daylight','Dawn/Dusk'],
    ['12-2 AM','2-4 AM','4-6 AM','6-8 AM','8-10 AM','10 AM - 12 PM','12-2 PM','2-4 PM','4-6 PM','6-8 PM','8-10 PM','10 PM - 12 AM'],
    ['Fog/Smoke/Smog','Cloudy','Clear','Blowing Sand/Soil/Snow','Severe Cross Wind','Snow','Rain','Sleet/Hail/Freezing Rain']]
b = list(itertools.product(*a))

In [42]:
len(b)

Out[72]: 9676800

In [43]:
from pyspark.sql import Row
rdd1 = sc.parallelize(b)
row_rdd = rdd1.map(lambda p: Row(Blocks=p[0],speedlim=p[1],weekday=p[2],light=p[3],timecollide=p[4],weather=p[5]))
allDF=sqlContext.createDataFrame(row_rdd)


In [44]:
allDF =allDF.withColumn("speedlim", allDF["speedlim"].cast(IntegerType()))

In [45]:
df_acc = result1.select(result1['Blocks'],result1['light'],result1['speedlim'],result1['timecollide'],result1['weather'],result1['weekday'])

In [46]:
df_acc.sort(col('frequency').desc()).show()

+------+--------+--------+-----------+-------+---------+
Blocks| light|speedlim|timecollide|weather| weekday|
+------+--------+--------+-----------+-------+---------+
 375|Daylight| 40| 4-6 PM| Clear| THURSDAY|
 375|Daylight| 40| 4-6 PM| Clear| FRIDAY|
 350|Daylight| 40| 4-6 PM| Clear| THURSDAY|
 375|Daylight| 40| 4-6 PM| Clear|WEDNESDAY|
 350|Daylight| 40| 4-6 PM| Clear| TUESDAY|
 350|Daylight| 40| 4-6 PM| Clear|WEDNESDAY|
 375|Daylight| 40| 4-6 PM| Clear| TUESDAY|
 374|Daylight| 40| 4-6 PM| Clear| TUESDAY|
 350|Daylight| 40| 4-6 PM| Clear| MONDAY|
 374|Daylight| 40| 4-6 PM| Clear| FRIDAY|
 350|Daylight| 40| 4-6 PM| Clear| FRIDAY|
 375|Daylight| 40| 2-4 PM| Clear| FRIDAY|
 374|Daylight| 40| 4-6 PM| Clear| MONDAY|
 375|Daylight| 40| 4-6 PM| Clear| MONDAY|
 573|Daylight| 40| 2-4 PM| Clear| FRIDAY|
 351|Daylight| 40| 4-6 PM| Clear| FRIDAY|
 350|Daylight| 40| 2-4 PM| Clear| FRIDAY|
 573|Daylight| 40| 4-6 PM| Clear| FRIDAY|
 662|Daylight| 30| 4-6 PM| Clear| TUESDAY|
 374|Daylight| 40| 4-6 PM| Clear|WEDNESDAY|
+------+--------+--------+-----------+-------+---------+
only showing top 20 rows

In [47]:
## Get the situations that accident did not occur

In [48]:
df_nonacc = allDF.subtract(df_acc)
#df_nonacc.count()

In [49]:
df_nonacc = df_nonacc.withColumn("frequency", lit(0))
#df_nonacc.show()

In [50]:
## Delete Files
#dbutils.fs.rm("/FileStore/aries_final.csv")

In [51]:
df_nonacc1 = df_nonacc.select("Blocks","speedlim","weekday","light","timecollide","weather","frequency")
#df_nonacc1.first()

In [52]:
df_nonacc1.printSchema()

root
-- Blocks: long (nullable = true)
-- speedlim: integer (nullable = true)
-- weekday: string (nullable = true)
-- light: string (nullable = true)
-- timecollide: string (nullable = true)
-- weather: string (nullable = true)
-- frequency: integer (nullable = false)

In [53]:
## Concatenate with df_groupby to get the final dataset
df_final = df_groupby.union(df_nonacc1)
#df_final

In [54]:
temp_table_name = "final"
df_final.createOrReplaceTempView(temp_table_name)
result2 = spark.sql("Select F.*,B.x1, B.x2,B.y1,B.y2,B.Xcentr,B.Ycentr from final as F, blockstable B where F.Blocks = B.Blocks")


+------+--------+---------+------------------+-------------+--------------------+---------+------------------+------------------+------------------+------------------+------+------+
Blocks|speedlim| weekday| light| timecollide| weather|frequency| x1| x2| y1| y2|Xcentr|Ycentr|
+------+--------+---------+------------------+-------------+--------------------+---------+------------------+------------------+------------------+------------------+------+------+
 26| 50| THURSDAY|Dark (Not Lighted)| 8-10 PM| Clear| 1|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 40| MONDAY|Dark (Not Lighted)| 6-8 AM| Clear| 1|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 40| SUNDAY| Dawn/Dusk| 4-6 AM| Rain| 1|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 50| TUESDAY|Dark (Not Lighted)| 6-8 PM| Rain| 1|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 60| FRIDAY|Dark (Not Lighted)| 4-6 PM| Clear| 1|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 30| FRIDAY|Dark (Not Lighted)|10 PM - 12 AM| Clear| 1|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 50| MONDAY|Dark (Not Lighted)| 8-10 PM| Clear| 1|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 40| MONDAY| Daylight|10 AM - 12 PM| Clear| 1|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 30| SUNDAY| Daylight| 2-4 PM|Blowing Sand/Soil...| 0|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 60| TUESDAY|Dark (Not Lighted)| 4-6 AM| Cloudy| 0|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 70| TUESDAY| Daylight| 4-6 AM|Sleet/Hail/Freezi...| 0|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 40|WEDNESDAY| Dark (Lighted)|10 AM - 12 PM|Blowing Sand/Soil...| 0|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 70| MONDAY|Dark (Not Lighted)| 4-6 PM| Clear| 0|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 60| THURSDAY| Dark (Lighted)| 8-10 PM|Sleet/Hail/Freezi...| 0|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 40| MONDAY| Dawn/Dusk| 8-10 PM|Blowing Sand/Soil...| 0|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 50|WEDNESDAY| Dawn/Dusk|10 PM - 12 AM|Blowing Sand/Soil...| 0|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 60| THURSDAY|Dark (Not Lighted)| 12-2 PM| Snow| 0|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 40| TUESDAY| Daylight| 4-6 AM| Cloudy| 0|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 50| MONDAY| Daylight| 8-10 PM| Cloudy| 0|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
 26| 50| MONDAY| Daylight| 8-10 PM|Sleet/Hail/Freezi...| 0|37.699999999999996|37.849999999999994|-88.08999999999999|-87.93999999999998| 37.8|-87.99|
+------+--------+---------+------------------+-------------+--------------------+---------+------------------+------------------+------------------+------------------+------+------+
only showing top 20 rows

In [55]:
#result1.toPandas().to_csv('/dbfs/FileStore/aries_accidents.csv')

In [56]:
result2.sort(col('frequency').desc()).show()